In [ ]:
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [ ]:
BASE_MODEL = "decapoda-research/llama-7b-hf"
 
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
 
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)
 
tokenizer.pad_token_id = (0)

tokenizer.padding_side = "left"